# Rockset

>[Rockset](https://rockset.com/) 是一个专为云构建的实时搜索和分析数据库。Rockset 使用 [Converged Index™](https://rockset.com/blog/converged-indexing-the-secret-sauce-behind-rocksets-fast-queries/) 和高效的向量嵌入存储，以大规模地提供低延迟、高并发的搜索查询。Rockset 完全支持元数据过滤，并处理持续更新的流数据的实时摄取。

本笔记本演示了如何将 `Rockset` 用作 LangChain 中的向量存储。开始之前，请确保您拥有 `Rockset` 账户和可用的 API 密钥。[立即开始免费试用。](https://rockset.com/create/)

您需要通过 `pip install -qU langchain-community` 安装 `langchain-community` 才能使用此集成。

## 设置您的环境

1. 利用 `Rockset` 控制台创建一个集合，并将 Write API 作为您的源。在本演练中，我们将创建一个名为 `langchain_demo` 的集合。
    
    配置以下 [ingest transformation](https://rockset.com/docs/ingest-transformation/) 来标记您的 embeddings 字段，并利用性能和存储优化：

   (在本示例中，我们使用了 OpenAI 的 `text-embedding-ada-002`，其中 #length_of_vector_embedding = 1536)

```
SELECT _input.* EXCEPT(_meta), 
VECTOR_ENFORCE(_input.description_embedding, #length_of_vector_embedding, 'float') as description_embedding 
FROM _input
```

2. 创建集合后，使用控制台检索 [API 密钥](https://rockset.com/docs/iam/#users-api-keys-and-roles)。在本笔记本中，我们假定您使用的是 `Oregon(us-west-2)` 区域。

3. 安装 [rockset-python-client](https://github.com/rockset/rockset-python-client)，以便 LangChain 能够直接与 `Rockset` 通信。

In [ ]:
%pip install --upgrade --quiet  rockset

## LangChain 教程

在您自己的 Python notebook 中进行操作，以生成向量嵌入并在 Rockset 中存储它们。
开始使用 Rockset 来搜索与您的搜索查询相似的文档。

### 1. 定义关键变量

In [ ]:
import os

import rockset

ROCKSET_API_KEY = os.environ.get(
    "ROCKSET_API_KEY"
)  # Verify ROCKSET_API_KEY environment variable
ROCKSET_API_SERVER = rockset.Regions.usw2a1  # Verify Rockset region
rockset_client = rockset.RocksetClient(ROCKSET_API_SERVER, ROCKSET_API_KEY)

COLLECTION_NAME = "langchain_demo"
TEXT_KEY = "description"
EMBEDDING_KEY = "description_embedding"

### 2. 准备文档

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Rockset
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

### 3. 插入文档

In [ ]:
embeddings = OpenAIEmbeddings()  # Verify OPENAI_API_KEY environment variable

docsearch = Rockset(
    client=rockset_client,
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    text_key=TEXT_KEY,
    embedding_key=EMBEDDING_KEY,
)

ids = docsearch.add_texts(
    texts=[d.page_content for d in docs],
    metadatas=[d.metadata for d in docs],
)

### 4. 搜索相似文档

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
output = docsearch.similarity_search_with_relevance_scores(
    query, 4, Rockset.DistanceFunction.COSINE_SIM
)
print("output length:", len(output))
for d, dist in output:
    print(dist, d.metadata, d.page_content[:20] + "...")

##
# output length: 4
# 0.764990692109871 {'source': '../../../state_of_the_union.txt'} Madam Speaker, Madam...
# 0.7485416901622112 {'source': '../../../state_of_the_union.txt'} And I’m taking robus...
# 0.7468678973398306 {'source': '../../../state_of_the_union.txt'} And so many families...
# 0.7436231261419488 {'source': '../../../state_of_the_union.txt'} Groups of citizens b...

### 5. 搜索相似文档并进行筛选

In [ ]:
output = docsearch.similarity_search_with_relevance_scores(
    query,
    4,
    Rockset.DistanceFunction.COSINE_SIM,
    where_str="{} NOT LIKE '%citizens%'".format(TEXT_KEY),
)
print("output length:", len(output))
for d, dist in output:
    print(dist, d.metadata, d.page_content[:20] + "...")

##
# output length: 4
# 0.7651359650263554 {'source': '../../../state_of_the_union.txt'} Madam Speaker, Madam...
# 0.7486265516824893 {'source': '../../../state_of_the_union.txt'} And I’m taking robus...
# 0.7469625542348115 {'source': '../../../state_of_the_union.txt'} And so many families...
# 0.7344177777547739 {'source': '../../../state_of_the_union.txt'} We see the unity amo...

### 6. [可选] 删除已插入的文档

您必须拥有与每个文档关联的唯一 ID，才能从您的集合中删除它们。
在插入文档时使用 `Rockset.add_texts()` 定义 ID。否则，Rockset 将为每个文档生成一个唯一的 ID。无论如何，`Rockset.add_texts()` 会返回已插入文档的 ID。

要删除这些文档，只需使用 `Rockset.delete_texts()` 函数。

In [ ]:
docsearch.delete_texts(ids)

## 摘要

在本教程中，我们成功创建了一个 `Rockset` 集合，`插入`了包含 OpenAI 嵌入的文档，并进行了带或不带元数据过滤器的相似文档搜索。

请关注 https://rockset.com/ 以获取该领域的未来更新。